## Tables to create

Key tables
* alignment
* death
* roles
* settings

Data tables
* player (just player name & id)
* game (general stats about each game)
* gms (gms for each game)
* playergame (each player's stats in each game)
* player_roles (roles for a row in playergame)

In [2]:
import pandas as pd 

In [3]:
# KEY TABLES

alignment = pd.read_csv("data/keys/alignment.csv").reset_index().rename(columns={"index": "alignment_id"})

print(alignment.head())

settings = pd.read_csv("data/keys/settings.csv")

print(settings.head())

death = pd.read_csv("data/keys/death.csv").reset_index().rename(columns={"index": "death_id"})

death.head()

   alignment_id alignment_char                     alignment_desc  is_elim  \
0             0              G                               Good    False   
1             1              E  Evil (Prim) (team kill / no conv)     True   
2             2              S             Secondary Evil Faction     True   
3             3              T              Tertiary Evil Faction     True   
4             4              ?            Evil Converter (w/kill)     True   

   is_evil  has_kill  has_convert  was_converted  
0    False     False        False          False  
1     True      True        False          False  
2     True      True        False          False  
3     True      True        False          False  
4     True      True         True          False  
           world  is_sanderson  is_cosmere
0  Cross-Cosmere          True        True
1            Sel          True        True
2       Scadrial          True        True
3        Nalthis          True        True
4         

,death_id,death_char,death_desc,dying_alignment,killer_alignment,non-kills
0,0,S,Survived,none,none,NaN
1,1,E,Killed by Eliminators,not E,E,NaN
2,2,X,Exed,any,none,NaN
3,3,V,Elim killed by Village,E,V,NaN
4,4,F,Villager Killed by Village (Friendly Fire),V,V,NaN


In [16]:
# IMPORTING DATA TAB
data = pd.read_csv("data/original_data/orig_data_040622.csv")

data.rename(inplace=True, columns={
    "ID": "pg_id",
    "Player": "player_name",
    "Alignment": "alignment_char",
    "Faction Outcome": "win", 
    "First Hit": "first_hit", 
    "Last Hit": "last_hit",
    "# of Hits": "num_hits",
    "Death/\n Survival": "death_char", # ???
    "Inactive": "inactive",
    "Mod": "IM",
    "Game Type": "game_format",
    "Game #": "game_num",
    "Unnamed: 15": "game_string", # these depend on other cols. name in sheet pls
    "Role": "role",
    "Unnamed: 18": "secondary_role",
    "Main?": "main_gm"
    })

data = data.filter([
    "pg_id", 
    "player_name",
    "alignment_char",
    "win",
    "first_hit", "last_hit", "num_hits",
    "death_char",
    "inactive",
    "GM", "main_gm", "IM", # "Spec"
    "game_string", #"game_format", "game_num",
    "role", "secondary_role"
    ])

data.head()

# more cleaning here? 

,pg_id,player_name,alignment_char,win,first_hit,last_hit,num_hits,death_char,inactive,GM,main_gm,IM,game_string,role,secondary_role
0,1,Meta,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,Y,NaN,LG1,GM,Roles that are unique and don't fit
1,2,Cracknut,G,L,1.0,1.0,1.0,E,NaN,NaN,NaN,NaN,LG1,Tineye,MR1/LG8/MR7 need work
2,3,Edgedancer,G,L,2.0,2.0,1.0,X,NaN,NaN,NaN,NaN,LG1,Cop,NaN
3,4,Mailliw73,G,L,4.0,4.0,1.0,X,NaN,NaN,NaN,NaN,LG1,Regular,NaN
4,5,Aether,E,W,4.0,4.0,1.0,V,NaN,NaN,NaN,NaN,LG1,Smoker,NaN


In [19]:
# IMPORTING GAME TAB

# header = 1 because there's an extra header row on top w categories
game = pd.read_csv("data/original_data/orig_game_040622.csv", header=1)

# cols wanted: id, format, number, string, anon num,
# mech balance, dist balance, IM id, start date, end date
# num cycles, num posts, setting id
# complexity, fundamentals, role madness

# cols still needed: format, number, mech balance, dist balance, 
# IM id, start date, end date, setting id
# IM id probably pulled from pg

game.reset_index(inplace=True)

game.columns

Index(['index', 'format', 'num', 'auto?', '# Players', '# Cycles', '# Posts',
       'Fundamentals', 'Winner', 'Complexity', 'Role Madness', 'Anon', 'Mech',
       'Dist', 'Broken', '# Elims', 'Elim %', '# Xed', '# Vig'd', '# Deaths',
       '# Survivors', '# E killed', '# M killed', '# N killed', '# V killed',
       '# FF', '# Xed.1', '# I killed', '# pinchhitters', '# Spectators',
       'GM(s)', 'IM', 'Title/Link', 'Sanderson? ', 'World', 'start_date',
       'end_date', 'link'],
      dtype='object')

In [20]:

# ! Need to be careful about sheet renaming cols - some kind of check?
game.rename(inplace=True, columns={
    "index": "game_id",
    "format": "game_format",
    "num": "game_num", 
    "auto?": "game_string", # optional? 
    #"# Players": "num_players", # optional
    "# Cycles": "num_cycles",
    "# Posts": "num_posts",
    "Fundamentals": "fundamentals", 
    "Role Madness": "role_madness",
    #"Winner": "winner", # optional
    "Complexity": "complexity",
    #"Broken": "broken", # TODO make into mech/dist once that's set in sheet
    "Mech": "mechanics_balance", # blank rn!!
    "Dist": "distribution_balance",
    "World": "setting", # need to make setting table and replace this w setting_id
    "Anon?": "anon_num", 
    "Title/Link": "title", # need to get link separately. hidden col? 
    # LINK
    "start_date": "start_date",
    "end_date": "end_date"
    })

print(game.columns)

# remaining columns are just calculated from data test
game = game.filter([
    "game_id",
    "game_format", "game_num", "game_string",
    # "num_players",
    #"IM", # currently blank - pulling from Data instead
    "num_cycles", 
    "num_posts", 
    "fundamentals", "role_madness", "complexity", 
    #"winner", 
    "mechanics_balance", "distribution_balance",
    #"broken", 
    "setting",
    "anon_num",
    "title",
    "start_date", "end_date"
])


game.tail()
# check for almost all NaN rows! extra checkboxes cause this

Index(['game_id', 'game_format', 'game_num', 'game_string', '# Players',
       'num_cycles', 'num_posts', 'fundamentals', 'Winner', 'complexity',
       'role_madness', 'Anon', 'mechanics_balance', 'distribution_balance',
       'Broken', '# Elims', 'Elim %', '# Xed', '# Vig'd', '# Deaths',
       '# Survivors', '# E killed', '# M killed', '# N killed', '# V killed',
       '# FF', '# Xed.1', '# I killed', '# pinchhitters', '# Spectators',
       'GM(s)', 'IM', 'title', 'Sanderson? ', 'setting', 'start_date',
       'end_date', 'link'],
      dtype='object')


,game_id,game_format,game_num,game_string,num_cycles,num_posts,fundamentals,role_madness,complexity,mechanics_balance,distribution_balance,setting,title,start_date,end_date
208,208,LG,83.0,LG83,6,NaN,V/E,False,NaN,NaN,NaN,NaN,The Survivor,NaN,NaN
209,209,BT,1.0,BT1,4,NaN,V/E,False,NaN,NaN,NaN,Cross-Cosmere,Auction of Stories,NaN,NaN
210,210,QF,59.0,QF59,7,NaN,V/E,False,NaN,NaN,NaN,Roshar,Bachelor - Roshar Edition!,NaN,NaN
211,211,MR,57.0,MR57,NaN,NaN,V/E,False,Semi Standard,NaN,NaN,NaN,NaN,NaN,NaN
212,212,LG,84.0,LG84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


END CLEANING 

# SQL table making

In [5]:
# PLAYER
unique_players = data["player_name"].unique()

player = pd.DataFrame({"player_id": range(len(unique_players)), "player_name": unique_players})

player.to_csv("data/player1.csv", index=False)

player.head()


,player_id,player_name
0,0,Meta
1,1,Cracknut
2,2,Edgedancer
3,3,Mailliw73
4,4,Aether


In [6]:
# SETTINGS, DEATH, ALIGNMENT

unique_worlds = game["world"].unique()

new_settings = []

for world in unique_worlds:
    if world not in settings["world"].unique():
        new_settings.append({"world": world, "is_sanderson": False, "is_cosmere": False})
        print(f"inserted {world}")

new_settings_df = pd.DataFrame(new_settings)

settings = pd.concat([settings, new_settings_df], ignore_index=True).reset_index().rename(columns={"index": "setting_id"})

settings.tail()

# TODO: check this against key table settings for worlds not in it
# also, don't have it insert NaN
# for each loc not there, put down as non-Sanderson and print that you're doing so

# do the same check with unique death and alignment tables, but don't insert by default - just print thing

inserted Hunger Games
inserted nan
inserted Game of Thrones
inserted KKC
inserted Death Note
inserted Codex Alera
inserted Pokemon
inserted Black Prism
inserted Red Rising
inserted Discworld
inserted Elder Scrolls
inserted My Little Pony
inserted Les Miserables
inserted Gunnerkrigg Court
inserted Lord of the Rings
inserted Kingkiller Chronicles
inserted Princess Bride
inserted Zelda
inserted Firefly


,setting_id,world,is_sanderson,is_cosmere
30,30,Lord of the Rings,False,False
31,31,Kingkiller Chronicles,False,False
32,32,Princess Bride,False,False
33,33,Zelda,False,False
34,34,Firefly,False,False


In [7]:
# checking death / alignment

al = data["alignment_char"].unique()

for a in al:
    if a not in alignment["alignment_char"].unique():
        print(a)



de = data["death_char"].unique()

for d in de:
    if d not in death["death_char"].unique():
        print(d)




nan
nan
L?
?
V/M
L
V/E


In [10]:
# TODO id replacements:
# in PG - player_id, game_id, alignment_id, death_id

merged_data = data.merge(player, on="player_name", how="left")

merged_data = merged_data.merge(game.filter(["game_id", "game_string"]), on="game_string", how="left")

merged_data = merged_data.merge(alignment.filter(["alignment_id", "alignment_char"]), on="alignment_char", how="left")

merged_data = merged_data.merge(death.filter(["death_id", "death_char"]), on="death_char", how="left")

merged_data.head()
# in game - setting_id

game = game.merge(settings, on="world", how="left")

In [12]:
# Pull rest of stuff into game table from PG: IM id, format/number(?)

IM_list = merged_data[merged_data["IM"] == "Y"].filter(["player_id", "game_id"])

games = game.merge(IM_list, on="game_id", how="left").rename(columns={"player_id": "IM_id"})

game.head()
# export games to csv (optionally)

,game_id,game_string,num_players,num_cycles,num_posts,fundamentals,role_madness,winner,complexity,broken,world,anon_num,title,setting_id,is_sanderson,is_cosmere
0,0,LG1,16.0,4,462.0,V/E,False,E,Standard,Well Balanced,Scadrial,False,In the Wake of Koloss,2,True,True
1,1,LG2,29.0,11,1232.0,CON,False,E,Standard,Balanced,Scadrial,False,Devil's Den,2,True,True
2,2,LG3,22.0,7,739.0,V/E,False,V,Standard,Broken Distribution,Scadrial,False,Blackwater Village,2,True,True
3,3,LG4,30.0,9,1029.0,V/E,False,V,Semi Standard,Broken Mechanics,Nalthis,False,Colours of War,3,True,True
4,4,QF1,12.0,4,118.0,V/E,False,E,Basic,Well Balanced,Sel,False,Jeskeri Mysteries,1,True,True


In [13]:
# make GMs table from PG

gms = merged_data[merged_data["GM"] == "Y"].filter(["player_id", "game_id"])

#gms.rename(columns={"player_id": "gm_id"}, inplace=True)

# ! MAIN GM

gms.head()

# optionally export gms table

,player_id,game_id
0,0,0.0
20,0,1.0
62,15,2.0
83,5,3.0
135,48,4.0


In [14]:
# if roles are in, make roles table

In [15]:
# clean PG - take out GM, spec, IM, ??


playergame = merged_data[merged_data["GM"] != "Y"]
playergame = playergame[playergame["IM"] != "Y"]
playergame = playergame[playergame["Spec"] != "Y"]

playergame = playergame.filter(['win', 'first_hit', 'last_hit', 'num_hits', 'inactive', 'Broken', 'player_id', 'game_id', 'alignment_id', 'death_id'])

# TODO: change Y -> True, nan -> False
# win W/L/D -> True/False/null

playergame = playergame.reset_index().rename(columns={"index": "pg_id"})


# optionally export playergame table

In [16]:
playergame.head()

,pg_id,win,first_hit,last_hit,num_hits,inactive,Broken,player_id,game_id,alignment_id,death_id
0,1,L,1.0,1.0,1.0,NaN,NaN,1,0.0,0.0,1.0
1,2,L,2.0,2.0,1.0,NaN,NaN,2,0.0,0.0,2.0
2,3,L,4.0,4.0,1.0,NaN,NaN,3,0.0,0.0,2.0
3,4,W,4.0,4.0,1.0,NaN,NaN,4,0.0,1.0,3.0
4,5,W,NaN,NaN,0.0,NaN,NaN,5,0.0,1.0,0.0


In [17]:
game.head() 
# can remove is_sanderson and is_cosmere

,game_id,game_string,num_players,num_cycles,num_posts,fundamentals,role_madness,winner,complexity,broken,world,anon_num,title,setting_id,is_sanderson,is_cosmere
0,0,LG1,16.0,4,462.0,V/E,False,E,Standard,Well Balanced,Scadrial,False,In the Wake of Koloss,2,True,True
1,1,LG2,29.0,11,1232.0,CON,False,E,Standard,Balanced,Scadrial,False,Devil's Den,2,True,True
2,2,LG3,22.0,7,739.0,V/E,False,V,Standard,Broken Distribution,Scadrial,False,Blackwater Village,2,True,True
3,3,LG4,30.0,9,1029.0,V/E,False,V,Semi Standard,Broken Mechanics,Nalthis,False,Colours of War,3,True,True
4,4,QF1,12.0,4,118.0,V/E,False,E,Basic,Well Balanced,Sel,False,Jeskeri Mysteries,1,True,True


In [18]:
alignment.head()

,alignment_id,alignment_char,alignment_desc,is_elim,is_evil,has_kill,has_convert,was_converted
0,0,G,Good,False,False,False,False,False
1,1,E,Evil (Prim) (team kill / no conv),True,True,True,False,False
2,2,S,Secondary Evil Faction,True,True,True,False,False
3,3,T,Tertiary Evil Faction,True,True,True,False,False
4,4,?,Evil Converter (w/kill),True,True,True,True,False


In [19]:
death.head()

,death_id,death_char,death_desc,dying_alignment,killer_alignment,non-kills
0,0,S,Survived,none,none,NaN
1,1,E,Killed by Eliminators,not E,E,NaN
2,2,X,Exed,any,none,NaN
3,3,V,Elim killed by Village,E,V,NaN
4,4,F,Villager Killed by Village (Friendly Fire),V,V,NaN


In [20]:
print(player.head())
print(gms.head())
print(settings.head())

   player_id player_name
0          0        Meta
1          1    Cracknut
2          2  Edgedancer
3          3   Mailliw73
4          4      Aether
     player_id  game_id
0            0      0.0
20           0      1.0
62          15      2.0
83           5      3.0
135         48      4.0
   setting_id          world  is_sanderson  is_cosmere
0           0  Cross-Cosmere          True        True
1           1            Sel          True        True
2           2       Scadrial          True        True
3           3        Nalthis          True        True
4           4         Roshar          True        True


## Tracking SQL appropriateness: 
* player: DONE
* setting: DONE  (remove nan row)
* alignment: maybe? 
* death: DONE (have extra cols not being inserted)
* **game**: nooo (missing some cols; bad types - see below)
* gms: DONE ish (not currently marking main GM bool, but SQL has a default and idc)
* **playergame**: nope (below)

### Problems with game 
* Missing columns: 
    - Game format & number (could pull in from pg, should put in sheet)
    - Mech and dist balance (not yet in sheet; just make all null? )
    - IM id (pull from pg)
    - anon_num (currently boolean)
    - start and end date (not yet in sheet)
    - link (not in sheet in a helpful way)
* Type problems: 
    - num_posts is a float (??)

### Problems with playergame 
* Type problems: 
    - first, last, and num hits are all floats and should be ints
    - win should be nullable boolean
    - ids for game, alignment, and death should be ints
    - inactive is Y/blank rather than t/f
* pinchhitter doesn't exist

### Where to check for bad characters:
* game.title 
* player.player_name
* game.fundamentals?

### Other
* PG needs to be cleaned to remove things not in death / alignment tables; want to be able to do constraints also on complexity / etc 

At this point should have the tables:
* playergame
* games (game)
* player
* gms
* alignment
* death
* settings

time for...
## SQL Insertion


In [20]:
import psycopg2
db = 1

ModuleNotFoundError: No module named 'psycopg2'

In [30]:
if db != 0:
    cnx = psycopg2.connect(user="isanford_123", database="isanford", host="localhost", password="12345678")
    cursor = cnx.cursor()


OperationalError: connection to server at "localhost" (127.0.0.1), port 5432 failed: FATAL:  password authentication failed for user "isanford"
connection to server at "localhost" (127.0.0.1), port 5432 failed: FATAL:  password authentication failed for user "isanford"


In [28]:
# PLAYER
#db = 0
cc = 0 

for row in player.iterrows():
    ins = "insert into player (player_id, player_name) values ({}, '{}');"
    ins = ins.format(row[1]["player_id"], row[1]["player_name"])

    if db == 1:
        #try: 
            cursor.execute(ins)
        # except:
        #     print(row)
        #     print(ins)
        #     cc = cc + 1

    else: 
        print(ins)
    
    break

print(f"Num errors: {cc}")

InsufficientPrivilege: permission denied for table player


In [24]:
alignment.head()

,alignment_id,alignment_char,alignment_desc,is_elim,is_evil,has_kill,has_convert,was_converted
0,0,G,Good,False,False,False,False,False
1,1,E,Evil (Prim) (team kill / no conv),True,True,True,False,False
2,2,S,Secondary Evil Faction,True,True,True,False,False
3,3,T,Tertiary Evil Faction,True,True,True,False,False
4,4,?,Evil Converter (w/kill),True,True,True,True,False


In [25]:
# KEY TABLES

# ALIGNMENT

cc = 0 

for row in alignment.iterrows():
    tup = row[1]
    ins = "INSERT INTO alignment (alignment_id, alignment_char, alignment_desc, is_elim, is_evil, has_kill, has_convert) VALUES('{}','{}','{}','{}','{}','{}','{}', '{}')"

    ins = ins.format(tup["alignment_id"], tup["alignment_char"], tup["alignment_desc"], tup["is_elim"], tup["is_evil"], tup["has_kill"], tup["has_convert"], tup["was_converted"])

    if db == 1:
        try: 
            cursor.execute(ins)
        except:
            print(tup)
            print(ins)
            cc = cc + 1

    else: 
        print(ins)
    break

print(f"Num errors: {cc}")

INSERT INTO alignment (alignment_id, alignment_char, alignment_desc, is_elim, is_evil, has_kill, has_convert) VALUES('0','G','Good','False','False','False','False', 'False')
Num errors: 0


In [26]:
# KEY TABLES

cc = 0 

# optional: also add other cols if they're reasonable

for row in death.iterrows():
    tup = row[1]
    ins = "INSERT INTO death (death_id, death_char, death_desc) VALUES('{}','{}','{}')"

    ins = ins.format(tup["death_id"], tup["death_char"], tup["death_desc"])

    if db == 1:
        try: 
            cursor.execute(ins)
        except:
            print(tup)
            print(ins)
            cc = cc + 1

    else: 
        print(ins)
    break

print(f"Num errors: {cc}")

INSERT INTO death (death_id, death_char, death_desc) VALUES('0','S','Survived')
Num errors: 0


In [27]:
settings.tail()

,setting_id,world,is_sanderson,is_cosmere
30,30,Lord of the Rings,False,False
31,31,Kingkiller Chronicles,False,False
32,32,Princess Bride,False,False
33,33,Zelda,False,False
34,34,Firefly,False,False


In [28]:
# KEY TABLES

cc = 0 

# optional: also add other cols if they're reasonable

for row in settings.iterrows():
    tup = row[1]
    ins = "INSERT INTO settings (setting_id, world, is_sanderson, is_cosmere) VALUES('{}','{}','{}','{}')"

    ins = ins.format(tup["setting_id"], tup["world"], tup["is_sanderson"], tup["is_cosmere"])

    if db == 1:
        try: 
            cursor.execute(ins)
        except:
            print(tup)
            print(ins)
            cc = cc + 1

    else: 
        print(ins)
        
    break

print(f"Num errors: {cc}")

INSERT INTO settings (setting_id, world, is_sanderson, is_cosmere) VALUES('0','Cross-Cosmere','True','True')
Num errors: 0


In [29]:
gms.head()

,player_id,game_id
0,0,0.0
20,0,1.0
62,15,2.0
83,5,3.0
135,48,4.0


In [30]:
# KEY TABLES

cc = 0 

# optional: also add other cols if they're reasonable

for row in gms.iterrows():
    tup = row[1]
    ins = "INSERT INTO gms (game_id, player_id) VALUES('{}','{}')"

    ins = ins.format(tup["game_id"], tup["player_id"])

    if db == 1:
        try: 
            cursor.execute(ins)
        except:
            print(tup)
            print(ins)
            cc = cc + 1

    else: 
        print(ins)
    break

print(f"Num errors: {cc}")

INSERT INTO gms (game_id, player_id) VALUES('0.0','0.0')
Num errors: 0


In [31]:
# KEY TABLES

cc = 0 

for row in game.iterrows():
    tup = row[1]
    ins = "INSERT INTO game (game_id, game_format, game_number, game_string, anon_num, mechanics_balance, distribution_balance, IM_id, start_date, end_date, num_cycles, num_posts, setting_id, complexity, fundamentals, role_madness) VALUES('{}','{}','{}','{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}')"

    ins = ins.format(
        tup["game_id"], 
        tup["game_format"], 
        tup["game_number"], 
        tup["game_string"], 
        tup["anon_num"],
        tup["mechanics_balance"], #not yet
        tup["distribution_balance"], # not yet
        tup["IM_id"],
        tup["start_date"], # not yet
        tup["end_date"], # not yet
        tup["num_cycles"], 
        tup["num_posts"],
        tup["setting_id"],
        tup["complexity"],
        tup["fundamentals"],
        tup["role_madness"] # ??
    )

    if db == 1:
        try: 
            cursor.execute(ins)
        except:
            print(tup)
            print(ins)
            cc = cc + 1

    else: 
        print(ins)
    break

print(f"Num errors: {cc}")

KeyError: 'game_format'

In [34]:
cc = 0 

for row in playergame.iterrows():
    tup = row[1]
    ins = "INSERT INTO playergame (pg_id, player_id, game_id, alignment_id, death_id, first_hit, last_hit, num_hits, win, pinchhitter, inactive) VALUES('{}','{}','{}','{}', '{}', '{}', '{}', '{}', '{}', '{}')" #, '{}')"

    ins = ins.format(
        tup["pg_id"], 
        tup["player_id"], 
        tup["game_id"], 
        tup["alignment_id"], 
        tup["death_id"],
        tup["first_hit"], 
        tup["last_hit"], 
        tup["num_hits"],
        tup["win"], 
        #tup["pinchhitter"], # not yet
        tup["inactive"]
    )

    if db == 1:
        try: 
            cursor.execute(ins)
        except:
            print(tup)
            print(ins)
            cc = cc + 1

    else: 
        print(ins)
    
    break

print(f"Num errors: {cc}")

INSERT INTO playergame (pg_id, player_id, game_id, alignment_id, death_id, first_hit, last_hit, num_hits, win, pinchhitter, inactive) VALUES('1','1','0.0','0.0', '1.0', '1.0', '1.0', '1.0', 'L', 'nan')
Num errors: 0
